In [98]:
import os
import json
import pandas as pd
import traceback


In [99]:
from langchain_ai21 import ChatAI21
import os
from dotenv import load_dotenv

In [127]:
KEY = os.getenv("AI21_API_KEY")
load_dotenv()

True

In [128]:
KEY

'dd3cb7dc-0bbf-437b-ad95-91382ccd1f37'

In [101]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [136]:
KEYS = 'dd3cb7dc-0bbf-437b-ad95-91382ccd1f37'

In [131]:
aillm = ChatAI21(api_key = KEY ,model="jamba-large",temperature=0.7)


In [132]:
response = aillm.invoke("Hi")
print(response)

[2025-03-21 16:16:10 - ai21.http_client.http_client - ERROR] Calling POST https://api.ai21.com/studio/v1 failed with a non-200 response code: 403 headers: Headers({'date': 'Fri, 21 Mar 2025 10:46:10 GMT', 'content-type': 'application/json', 'content-length': '43', 'connection': 'keep-alive', 'request-id': '2736257e-d5fc-3a03-24bb-ccaa783eedd9', 'via': '1.1 google', 'cf-cache-status': 'DYNAMIC', 'strict-transport-security': 'max-age=15552000; includeSubDomains', 'expect-ct': 'max-age=86400, enforce', 'referrer-policy': 'same-origin', 'x-content-type-options': 'nosniff', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'server': 'cloudflare', 'cf-ray': '923cdd27db2354d4-DEL'})


AI21APIError: Failed with http status code: 403 (AI21APIError). Details: {"detail":"Access denied for this feature"}

In [144]:
from ai21 import AI21

client = AI21(api_key='dd3cb7dc-0bbf-437b-ad95-91382ccd1f37')  # or pass it in directly

response = client.chat.completions.create(
    model='jamba-large',
    messages=[{'role': 'user', 'content': 'Hello, please provide a brief explanation of what AI21 does, in 100 words or less"'}]
)

print(response)


TypeError: 'NoneType' object is not callable

In [103]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [104]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your task to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [105]:
quiz_template = PromptTemplate(
    input_variables=["text","number","subject","tone"],
    template= TEMPLATE
)

In [106]:
quiz_chain=LLMChain(llm=aillm, prompt=quiz_template, output_key="quiz", verbose=True)

In [107]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [108]:
quiz_evaluation_template=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [109]:
review_chain=LLMChain(llm=aillm, prompt=quiz_template, output_key="review", verbose=True)

In [110]:
evaluation_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [111]:

file_path=r"C:\Users\jainv\mcqgen\data.txt"

In [112]:
with open(file_path,"r") as file:
    TEXT = file.read()

In [113]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [114]:

response = evaluation_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defin

[2025-03-21 16:11:01 - ai21.http_client.http_client - ERROR] Calling POST https://api.ai21.com/studio/v1 failed with a non-200 response code: 403 headers: Headers({'date': 'Fri, 21 Mar 2025 10:41:02 GMT', 'content-type': 'application/json', 'content-length': '43', 'connection': 'keep-alive', 'request-id': 'd3eaffae-6e0d-ff49-9755-531956c3bebe', 'via': '1.1 google', 'cf-cache-status': 'DYNAMIC', 'strict-transport-security': 'max-age=15552000; includeSubDomains', 'expect-ct': 'max-age=86400, enforce', 'referrer-policy': 'same-origin', 'x-content-type-options': 'nosniff', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'server': 'cloudflare', 'cf-ray': '923cd59f1f055484-DEL'})


AI21APIError: Failed with http status code: 403 (AI21APIError). Details: {"detail":"Access denied for this feature"}

In [ ]:
print(response.get("quiz"))

NameError: name 'response' is not defined

In [ ]:
if isinstance(quiz, str):  
    quiz = quiz.strip()
    print(quiz)

        

NameError: name 'quiz' is not defined

In [ ]:

print(quiz)

n


In [ ]:


quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

AttributeError: 'str' object has no attribute 'items'

In [ ]:
print(quiz_table_data)

[]


In [ ]:
quiz_new

NameError: name 'quiz_new' is not defined

In [ ]:
import json

def is_json_serializable(obj):
    try:
        json.dumps(obj)  # Try converting it to JSON
        return True
    except (TypeError, OverflowError):
        return False

RESPONSE_JSON = {"key": "value"}  # Example JSON

if is_json_serializable(RESPONSE_JSON):
    json_string = json.dumps(RESPONSE_JSON)
else:
    print("Error: RESPONSE_JSON contains non-serializable data!")


In [ ]:
is_json_serializable(quiiz)

True